In [44]:
import pandas as pd
import os
import shutil
from collections import defaultdict

In [45]:
tracks = pd.read_csv("data/audio/fma_metadata/tracks.csv")
track_info = tracks[["track.7",'track.8']]
track_info.columns = track_info.iloc[0].rename('track_id')
track_info = track_info.iloc[2:]

/tmp/ipykernel_2566712/2881960075.py:1: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv("data/audio/fma_metadata/tracks.csv")


In [46]:
track_info.genre_top.value_counts()

genre_top
Rock                   14182
Experimental           10608
Electronic              9372
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: count, dtype: int64

In [47]:
track_info.genres.value_counts()

genres
[21]                    2735
[15]                    2689
[]                      2231
[12]                    1896
[1, 30, 38, 41, 247]    1633
                        ... 
[15, 240, 468]             1
[17, 63, 1235]             1
[10, 12, 169]              1
[25, 33]                   1
[13, 763]                  1
Name: count, Length: 4768, dtype: int64

In [48]:
file_list = []
for root, dirs, files in os.walk("data/audio/fma_small"):
    for file in files:
        file_list.append((os.path.join(root, file), file.replace(".mp3", "").lstrip("0")))

file_list[:5]

[('data/audio/fma_small/139/139776.mp3', '139776'),
 ('data/audio/fma_small/139/139043.mp3', '139043'),
 ('data/audio/fma_small/139/139110.mp3', '139110'),
 ('data/audio/fma_small/139/139934.mp3', '139934'),
 ('data/audio/fma_small/139/139935.mp3', '139935')]

In [49]:
# def get_genre_top(file_id, track_info_dict):
#     return track_info_dict.get(int(file_id), None)

# track_info_dict = track_info['genre_top'].to_dict()

# result = [(file[1], get_genre_top(file[1], track_info_dict), file[0]) for file in file_list]
# result[:5]

In [50]:
def reorganize_by_genre(result, destination_folder="data/audio/fma_small_by_genre"):
    os.makedirs(destination_folder, exist_ok=True)
    
    # 用于记录每个风格的文件数量
    genre_count = defaultdict(int)
    
    for file_id, genre, file_path in result:
        # 如果风格为空（nan），跳过这个文件
        if pd.isna(genre):
            continue

        # 创建以风格为名的文件夹
        genre_folder = os.path.join(destination_folder, genre)
        os.makedirs(genre_folder, exist_ok=True)

        # 移动文件到对应的风格文件夹
        file_name = os.path.basename(file_path)
        new_path = os.path.join(genre_folder, file_name)
        shutil.move(file_path, new_path)
        
        # 增加对应风格的文件计数
        genre_count[genre] += 1
        # print(f"Moved {file_path} to {new_path}")
        
    print("\nSong count by genre:")
    for genre, count in genre_count.items():
        print(f"{genre}: {count} files")

In [51]:
# reorganize_by_genre(result)

```
Song count by genre:
Electronic: 394 files
Experimental: 455 files
Rock: 645 files
Folk: 105 files
Instrumental: 83 files
Country: 14 files
International: 54 files
Classical: 53 files
Hip-Hop: 129 files
Pop: 114 files
Spoken: 27 files
Old-Time / Historic: 30 files
Soul-RnB: 4 files
Jazz: 17 files
Easy Listening: 2 files
Blues: 1 files
```

In [52]:
def count_songs_by_genre(destination_folder="data/spec/fma_small"):
    genre_count = defaultdict(int)
    
    # 遍历目标文件夹中的每个风格文件夹
    for genre_folder in os.listdir(destination_folder):
        genre_path = os.path.join(destination_folder, genre_folder)
        
        # 确保该路径是一个文件夹
        if os.path.isdir(genre_path):
            # 统计该风格文件夹中的文件数
            genre_count[genre_folder] = len(os.listdir(genre_path))
    
    # 输出每个风格的文件数量
    print("Song count by genre:")
    for genre, count in genre_count.items():
        print(f"{genre}: {count} files")

# count_songs_by_genre()